In [1]:
import cmipld
from cmipld.utils.offline import LD_server, replace_loader
from pyld import jsonld

/Users/daniel.ellis/customlib/homebrew/Caskroom/mambaforge/base/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Set up offline override for the current repository

ldpath = cmipld.utils.git.ldpath()


# create a temporary directory containing (a copy of) the current repository
# localserver = LD_server(copy = [[ldpath,'current_repo']])
repos= { 
        'https://wcrp-cmip.github.io/WCRP-universe/': 'universal',
        'https://wcrp-cmip.github.io/MIP-variables/': 'variables',
        'https://wcrp-cmip.github.io/CMIP6Plus_CVs/': 'cmip6plus'
        }

localserver = LD_server(repos = repos.items(),copy = [[ldpath,'cmip6plus']])

# localserver = LD_server(copy = [[ldpath,'cmip6plus']])


localhost = localserver.start_server(8082)
repo_url = cmipld.utils.git.url()
io_url = cmipld.utils.git.url2io(repo_url)





https://wcrp-cmip.github.io/WCRP-universe/ universal /var/folders/hc/s_7lggq12nndglbdyrn3f91m1l58yd/T/cmipld_local_08zezpe72025-01-16T10:53:39


Cloning into '/var/folders/hc/s_7lggq12nndglbdyrn3f91m1l58yd/T/cmipld_local_08zezpe72025-01-16T10:53:39/universal'...




https://wcrp-cmip.github.io/MIP-variables/ variables /var/folders/hc/s_7lggq12nndglbdyrn3f91m1l58yd/T/cmipld_local_08zezpe72025-01-16T10:53:39


Cloning into '/var/folders/hc/s_7lggq12nndglbdyrn3f91m1l58yd/T/cmipld_local_08zezpe72025-01-16T10:53:39/variables'...




https://wcrp-cmip.github.io/CMIP6Plus_CVs/ cmip6plus /var/folders/hc/s_7lggq12nndglbdyrn3f91m1l58yd/T/cmipld_local_08zezpe72025-01-16T10:53:39


Cloning into '/var/folders/hc/s_7lggq12nndglbdyrn3f91m1l58yd/T/cmipld_local_08zezpe72025-01-16T10:53:39/cmip6plus'...




Repositories cloned into <TemporaryDirectory '/var/folders/hc/s_7lggq12nndglbdyrn3f91m1l58yd/T/cmipld_local_08zezpe72025-01-16T10:53:39'>
Copying the repo into LocalServer  /Users/daniel.ellis/WIPwork/CMIP6Plus_CVs/src-data/ --> cmip6plus
Repositories copied into <TemporaryDirectory '/var/folders/hc/s_7lggq12nndglbdyrn3f91m1l58yd/T/cmipld_local_08zezpe72025-01-16T10:53:39'>


......+......+.....+++++++++++++++++++++++++++++++++++++++++++++*..+++++++++++++++++++++++++++++++++++++++++++++*.......+.+..+......+..........+.................+..................+.........+.......+........+.......+...+......+..+.......+....................+......+....+..+.+.....+.............+.................+....+...+....................+...+...+.......+..+.............+..+......+.+..............+......+....+..+............+............+.........+............+....+.....+....+............+..+.+..+.......+..+.........+....+..+.+......................................+....+........+...+..........+.................+.......+.....+.......+...+..+..................+.+......+...+........+.............+......+...+...+........+.........+...............+...+.........+..............................+....+..+............+............+...+.......+....................+...+......+..........+..+.......+..+.........................+.........+......+...+.....+............+.+...+.....+.+..............+.

Created SSL certificates in: /var/folders/hc/s_7lggq12nndglbdyrn3f91m1l58yd/T/cmipld_local_08zezpe72025-01-16T10:53:39
Serving /var/folders/hc/s_7lggq12nndglbdyrn3f91m1l58yd/T/cmipld_local_08zezpe72025-01-16T10:53:39 at https://localhost:8082
make test for url2io


............+.....+.......+..+.+........+..........+..+.........+...+...+.........+.........+.......+...+..+.........+......+.............+.....+....+.....+......................+...+...........+..........+..+..........+..............+...+..........+......+...........+.........+.+...........+...+......+.+.....+...+.+...+..+....+..+...+.........+................+......+...+.....+...+....+.....+...................+......+........+......+.+......+...+.........+..+...+..........+.....+......+.+++++
-----


In [4]:
# 
replace_loader(localhost,[list(i) for i in repos.items()])


<function requests_document_loader.<locals>.loader at 0x10da07b50>
Setting up location forwarding for:
 -  https://wcrp-cmip.github.io/WCRP-universe/ >>> https://localhost:8082/universal/
 -  https://wcrp-cmip.github.io/MIP-variables/ >>> https://localhost:8082/variables/
 -  https://wcrp-cmip.github.io/CMIP6Plus_CVs/ >>> https://localhost:8082/cmip6plus/



In [3]:


def get_entry(data,entry='label'):
    return [i.get(entry) for i in data]

def tables(data):
    return get_entry(data['tables'])
    
# getattr(cmipld,'processor')

In [5]:
# getattr(cmipld,'processor')

In [ ]:
from collections import OrderedDict

CV = OrderedDict()


for item in [
    'product',
    'mip-era',
    'license',
    'activity',
    'tracking-id',
    'drs',
    'repo',
    # 'tables',
    'required-global-attributes',
    'index'
    ]:
    
    print(item)
    # CV[item] = getattr(locals(),item)
    CV[item] = cmipld.processor.get(f'{io_url}project/{item}.json', depth=3)[0]



In [ ]:
CV['product']

In [ ]:
repo_url = cmipld.utils.git.url()
io_url = cmipld.utils.git.url2io(repo_url)

In [ ]:


# cmipld.processor.loader(f'{io_url}project/_context_',{})

jsonld.expand([{
        "@context": [
        {
            "@base": "https://wcrp-cmip.github.io/CMIP6Plus_CVs/project/",
            "@vocab": "https://wcrp-cmip.github.io/CMIP6Plus_CVs/project/",
            "activity": {
                "@container": "@id",
                "@context": "http://wcrp-cmip.github.io/WCRP-universe/activity/_context_"
            },
            "mip_era": {
                "@type": "@id",
                "@context": "http://wcrp-cmip.github.io/WCRP-universe/mip/_context_"
            },
            "product": {
                "@type": "@id",
                "@context": "https://wcrp-cmip.github.io/WCRP-universe/product/_context_"
            },
            "tables": {
                "@type": "@id",
                "@context": "http://wcrp-cmip.github.io/MIP-variables/tables/_context_"
            },
            "license":{
                "@type": "@id",
                "@context": "http://wcrp-cmip.github.io/WCRP-universe/license/_context_"
            }
        }
    ],
        '@id': 'project/product',
  'product': ['model-output'],
  '@type': ['wcrp:project',
   'https://wcrp-cmip.github.io/CMIP6Plus_CVs/',
   'wcrp:product']}])#,f'{io_url}project/product.json')



In [ ]:
import requests
item='product'
print(io_url)
iourl = localhost
d = requests.get(f'{io_url}project/{item}.json').json()
c = requests.get(f'{io_url}project/_context_').json()
c

In [ ]:
d['@context'] = c['@context'][1]
jsonld.expand(d)

In [5]:
item='product'
jsonld.expand(f'{io_url}project/{item}.json')

127.0.0.1 - - [16/Jan/2025 10:54:01] "GET /cmip6plus/project/product.json HTTP/1.1" 200 -
127.0.0.1 - - [16/Jan/2025 10:54:01] "GET /universal/activity/_context_ HTTP/1.1" 200 -
127.0.0.1 - - [16/Jan/2025 10:54:01] "GET /universal/_context_ HTTP/1.1" 200 -
127.0.0.1 - - [16/Jan/2025 10:54:01] "GET /universal/mip/_context_ HTTP/1.1" 200 -
127.0.0.1 - - [16/Jan/2025 10:54:01] "GET /universal/product/_context_ HTTP/1.1" 200 -
127.0.0.1 - - [16/Jan/2025 10:54:01] "GET /variables/tables/_context_ HTTP/1.1" 200 -
127.0.0.1 - - [16/Jan/2025 10:54:01] "GET /variables/_context_ HTTP/1.1" 200 -
127.0.0.1 - - [16/Jan/2025 10:54:01] "GET /variables/variables/_context_ HTTP/1.1" 200 -
127.0.0.1 - - [16/Jan/2025 10:54:03] "GET /universal/license/_context_ HTTP/1.1" 200 -


[{'https://wcrp-cmip.github.io/CMIP6Plus_CVs/project/id': [{'@value': 'product'}],
  'https://wcrp-cmip.github.io/CMIP6Plus_CVs/project/product': [{'@value': 'model-output'}],
  'https://wcrp-cmip.github.io/CMIP6Plus_CVs/project/type': [{'@value': 'wcrp:project'},
   {'@value': 'cmip6plus'},
   {'@value': 'wcrp:product'}]}]

In [ ]:
# This means we can make 'locally hosted' content appear as an external resource in our PYLD instance

testfile = f'{io_url}project/'



In [ ]:
CV['license']

In [ ]:
item = 'tables'




frame = {
    '@context': [
   {'@base': 'https://wcrp-cmip.github.io/CMIP6Plus_CVs/project/',
    '@vocab': 'https://wcrp-cmip.github.io/CMIP6Plus_CVs/project/',
    'activity': {'@container': '@id',
     '@context': 'https://wcrp-cmip.github.io/WCRP-universe/activity/_context_'},
    'mip_era': {'@type': '@id',
     '@context': 'https://wcrp-cmip.github.io/WCRP-universe/mip/_context_'},
    'product': {'@type': '@id',
     '@context': 'https://wcrp-cmip.github.io/WCRP-universe/product/_context_'},
    'tables': {'@type': '@id',
     '@context': 'https://wcrp-cmip.github.io/MIP-variables/tables/_context_'},
    'license': {'@type': '@id',
     '@context': 'https://wcrp-cmip.github.io/WCRP-universe/license/_context_'}}],
    "id": "product",
    "type": [
        "wcrp:project",
        "cmip6plus",
        "wcrp:product"
    ],
    "product": [
        "model-output"
    ]
         
}
         
         
        #  ,"tables":[{"product":""}],"@embed":"@always"}

res = jsonld.expand(frame)

res

In [ ]:
#  if depth = 1 parallel, otherwise serial 
#   ptqdm?


cahcheing 
fail repeat 



In [ ]:
from p_tqdm import p_map

In [ ]:
from pyld import jsonld
jsonld.frame(cmipld.get(f'{io_url}project/{item}.json'), {
    '@context': [
        f'{io_url}project/{item}.json', 
        ],
    "tables":{
            "table_id":'',
            "@explicit": True,
            }
    })

In [ ]:
jsonld.frame(f'{cmipld.mapping["variables"]}tables/apsubhrptsite',{"@context":[f'{cmipld.mapping["variables"]}tables/apsubhrptsite',{}],"table_id":{},"@explicit":True})

In [ ]:
# localserver.copy_existing_repos([[ldpath,'current_repo']])

In [ ]:
cmipld.get(f'{io_url}source/cesm2.json')

In [ ]:
           "model-component": {
                "@context": "https://wcrp-cmip.github.io/CMIP6Plus_CVs/component/_context_",
                "@type": "@id"
            }

In [ ]:
jsonld.expand(f'{io_url}source/cesm2.json')